In [2]:
def readDataFrameFromCloudant(host,user,pw,database):
    sparkSession = SQLContext.getOrCreate(sc).sparkSession

    cloudantdata=sparkSession.read.format("com.cloudant.spark"). \
    option("cloudant.host",host). \
    option("cloudant.username", user). \
    option("cloudant.password", pw). \
    load(database)

    cloudantdata.createOrReplaceTempView("washing")
    spark.sql("SELECT * from washing").show()
    return cloudantdata

In [3]:
#TODO Please provide your Cloudant credentials here
hostname = "thisIsTheHostNameOfYourCloudantServiceInBluemix"
user = "thisIsTheUserNameOfYourCloudantServiceInBluemix"
pw = "thisIsThePasswordOfYourCloudantServiceInBluemix"
database = "washing" #as long as you didn't change this in the NodeRED flow the database name stays the same
cloudantdata=readDataFrameFromCloudant(hostname, user, pw, database)

+--------------------+--------------------+-----+--------+----------+---------+--------+-----+-----------+-------------+-------+
|                 _id|                _rev|count|flowrate|fluidlevel|frequency|hardness|speed|temperature|           ts|voltage|
+--------------------+--------------------+-----+--------+----------+---------+--------+-----+-----------+-------------+-------+
|0007568c977866188...|1-f5c43083af9779c...|   95|      11|acceptable|     null|      75| null|        100|1489699043697|   null|
|0007568c977866188...|1-7feb8945c3d4e0b...|   32|    null|      null|       62|    null| null|       null|1489699044417|    224|
|0007568c977866188...|1-3f94674697d81b9...|   33|    null|      null|       74|    null| null|       null|1489699047419|    220|
|0007568c977866188...|1-24ae3e443885f51...|   34|    null|      null|       78|    null| null|       null|1489699050422|    235|
|0007568c977866188...|1-a35c5dc8dd08737...|  141|      11|acceptable|     null|      74| null|   

In [3]:
# install latest local version
!pip install pixiedust

In [4]:
from pixiedust.display import *
display(cloudantdata)

In [ ]:
spark.sql("select avg(voltage), stdev(voltage), max(voltage) from washing").show()

In [9]:
df = spark.sql("""
select mean(voltage) as mean, std(voltage) as std from (
    SELECT ROW_NUMBER() OVER (ORDER BY ts desc) AS row, * FROM washing where voltage is not null
) where
row < 30
""")
               
df.show()
mean = df.first().mean
std = df.first().std
spark.sql("SELECT ({0}-voltage)/({1}+0.0001) as zscore, voltage FROM washing where voltage is not null order by ts desc".format(mean,std)).collect()

+-----------------+------------------+
|             mean|               std|
+-----------------+------------------+
|229.3793103448276|6.5269430227078615|
+-----------------+------------------+



[Row(zscore=Decimal('0.6385450632388'), voltage=225),
 Row(zscore=Decimal('1.0901988884932'), voltage=222),
 Row(zscore=Decimal('-0.5658651374396'), voltage=233),
 Row(zscore=Decimal('-1.4691727879484'), voltage=239),
 Row(zscore=Decimal('0.9396476134084'), voltage=223),
 Row(zscore=Decimal('0.1868912379844'), voltage=228),
 Row(zscore=Decimal('1.2407501635780'), voltage=221),
 Row(zscore=Decimal('1.0901988884932'), voltage=222),
 Row(zscore=Decimal('0.6385450632388'), voltage=225),
 Row(zscore=Decimal('-1.0175189626940'), voltage=236),
 Row(zscore=Decimal('-0.1142113121852'), voltage=230),
 Row(zscore=Decimal('0.0363399628996'), voltage=229),
 Row(zscore=Decimal('-1.4691727879484'), voltage=239),
 Row(zscore=Decimal('1.0901988884932'), voltage=222),
 Row(zscore=Decimal('-1.4691727879484'), voltage=239),
 Row(zscore=Decimal('-1.3186215128636'), voltage=238),
 Row(zscore=Decimal('-0.7164164125244'), voltage=234),
 Row(zscore=Decimal('-0.7164164125244'), voltage=234),
 Row(zscore=Decimal

In [11]:
import requests
r = requests.get("https://1234thisismyname.mybluemix.net/edgemodelupdate?command=emergencyshutdown")
r.status_code

200